In [1]:
import pandas as pd

In [2]:
# read data (encoded with word2vec) as df
id_lyrics_word2vec_data = pd.read_csv('/content/id_lyrics_word2vec_mmsr.tsv', sep='\t')
info = pd.read_csv('/content/id_information_mmsr.tsv', sep='\t')
id_lyrics_word2vec_data

,id,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,vo7FwIGCk6YHfkIA,0.033718,0.014378,0.026128,0.042627,-0.048268,0.015804,0.032547,-0.020543,0.025851,...,-0.018858,0.016099,-0.040894,-0.005447,-0.016220,-0.014547,0.017679,-0.040378,-0.015388,-0.003768
1,KbiCjEDMzzHYn6zF,0.032313,0.013010,0.024825,0.028559,-0.024753,-0.007209,0.018488,-0.037101,0.031233,...,-0.044380,0.023839,-0.051918,0.023653,-0.000048,0.011242,0.023039,-0.017589,0.015194,-0.017180
2,pzQHBx7Y3N1uVlpT,0.030761,0.016845,0.019547,0.045954,-0.022321,-0.002361,0.037860,-0.028647,0.029722,...,-0.042202,0.037827,-0.035356,0.017195,-0.032505,-0.011599,-0.000328,-0.034397,-0.002424,-0.022258
3,xwJc7oM2dkbh17Sy,0.045185,0.013670,0.025823,0.032239,-0.008347,-0.015055,0.018252,-0.024198,0.032537,...,-0.004318,0.021038,-0.059454,0.025127,-0.013767,-0.015213,-0.002923,-0.024353,0.008792,-0.004122
4,Gh4yUdMJrPrPAJjb,0.017956,0.018557,0.020527,0.042157,-0.019683,-0.004840,0.021483,-0.030211,0.023600,...,-0.014065,0.037840,-0.047416,-0.011087,-0.024351,-0.012404,0.009918,-0.028100,0.011513,0.001832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10090,tEooHa0Wo0X7tjSl,-0.000929,0.035526,0.027385,0.027430,0.017913,-0.013159,0.026053,-0.051103,0.017601,...,0.009480,0.018184,-0.045289,0.006032,-0.029483,-0.034000,0.000721,-0.027399,0.015489,-0.011772
10091,Aor1UfyURwiyPLbf,0.009526,0.011349,0.005828,0.035367,-0.008414,-0.011917,0.003640,-0.034490,0.021491,...,-0.017593,0.021131,-0.046282,-0.001318,-0.011933,-0.016869,-0.001362,-0.014855,-0.001428,-0.000143
10092,ce0oJuGClEKx1p5h,0.020440,0.020249,0.003471,0.027390,-0.034882,0.001986,0.032323,-0.052257,0.037561,...,-0.044087,0.025566,-0.045234,-0.004285,-0.019626,-0.017820,0.011638,-0.011743,0.024890,-0.008813
10093,YmpeYhRyOUlU6AWJ,0.021737,0.019791,0.011143,0.022635,-0.014265,-0.000437,0.026872,-0.031717,0.029828,...,-0.023752,0.007710,-0.027778,0.000604,-0.007336,-0.004684,-0.006033,-0.012436,0.010231,0.003211


In [3]:
def get_song_info_from_ids(track_ids, info_dataset):
    song_info = []
    for track_id in track_ids:
        info_entry = info_dataset[info_dataset['id'] == track_id]
        if not info_entry.empty:
            song_info.append((info_entry.iloc[0]['song'], info_entry.iloc[0]['artist']))
    return song_info

In [4]:
def cosim(query_ids,data,N,feature):
  'data: (df): a dataframe of the data features (i.e: word2vec...)'
  'query_ids: (list) list of the song ids to drop from the original data'
  'N: (int) number of track ids to extract'
  'feature: (str) the feature/mode name used (i.e: word2vec) '
  from sklearn.metrics.pairwise import cosine_similarity
  from tqdm import tqdm


  #  drop query songs from the data
  updated_data = data.drop(query_ids)
  # list of the songs to use as query
  query_songs = [data.iloc[i,1:] for i in query_ids]
  # print(query_songs )

  # calculate similarities
  similarities = {} # dict that will contain all song id queries as key accompanied by the 10 similarities list as value
  for id, query in zip(query_ids,query_songs):
    cosim_list = [] # list that will contain all the similarities between the querry and each document in the data
    try :
      for index, doc in (updated_data.drop(columns=['id']).iterrows()):
        cosim_list.append((updated_data.iloc[index]['id'],cosine_similarity(query.values.reshape(1,-1),doc.values.reshape(1,-1))))
    except Exception as e:
      print('error --->',e)

    cosim_list.sort(key=lambda x: x[1], reverse=True)
    similarities[f'query_id_{id}'] = cosim_list[:N]

  for query in similarities:
    print(f'The Most similar track IDs for song {query} using {feature} feature: ',[similarities[query][i][0] for i in range(len(similarities[query]))])

  return similarities

In [5]:
sim = cosim([0,1,2],id_lyrics_word2vec_data,10,'Word2vec')

error ---> single positional indexer is out-of-bounds
error ---> single positional indexer is out-of-bounds
error ---> single positional indexer is out-of-bounds
The Most similar track IDs for song query_id_0 using Word2vec feature:  ['khWdlbyBXH5sXHBg', 'd3qNK3tkDuX6acop', 'OzJxZkIY1Hsgb6sY', '1CCSNgcHsP1fqhiE', 'NBcHwrxNc4PYOk5b', 'yZPLE6qF7hqSAzNE', 'L7fA9PDmAwO8bPV8', 'hrhygDaIawUqTCyz', '7ak0DgWafWwGo4T6', '4grnFRYaFqqWlKXE']
The Most similar track IDs for song query_id_1 using Word2vec feature:  ['ui7NivP1oQ9Elx5c', 'AfQy4d8uFoPHysbN', '98iaNi4kVKEjLJZC', 'tPNrNoLg1TH2zQ0A', 'fl8040L0RnNLSqrr', 'BPP5MYvFRLEqWe83', 'h2ap0A9dSDkFmO8D', 'W3CCtzRjme0zjren', 'OKUKjt4wOOtAkShf', '4EELClH71ydoTa7D']
The Most similar track IDs for song query_id_2 using Word2vec feature:  ['rUn4jfoR0KrYBN6O', '7iUOzBjgyJlTlg9y', 'Q837q2LPqffyecBB', '4tcihoDYX1dMfZPp', 'utRMaNxaE60bJazF', 'XZEzDv414B7hP35u', 'pENgorWdXpxA7bac', 'ZYn6aDv71F4oaN73', 'duZJP3I5NQEoL95B', 'phE7ePEvY2LXoGj0']


In [6]:
# Collect the IDs of the most similar tracks
for query in sim:
  most_similar_track_ids = []
  print(f'\n######################  QUERY: {query} ################################\n')
  for song_id, similarity in sim[query]:
      most_similar_track_ids.append(song_id)

  song_info = get_song_info_from_ids(most_similar_track_ids, info)

  # Print the song and artist information
  for song, artist in song_info:
      print(f"Song: {song}, Artist: {artist}")



######################  QUERY: query_id_0 ################################

Song: The Last Ship, Artist: Sting
Song: Life's Like A River, Artist: Scorpions
Song: Axe Of Judgement, Artist: Ensiferum
Song: Perfect Blue Buildings, Artist: Counting Crows
Song: The Bitter End, Artist: Insomnium
Song: Lance Jr., Artist: Courtney Barnett
Song: Burst, Artist: Magazine
Song: Nothing, Artist: Janet Jackson
Song: In the Belly of a Shark, Artist: Gallows
Song: Life Goes On, Artist: LeAnn Rimes

######################  QUERY: query_id_1 ################################

Song: I can't quit you baby, Artist: The Rolling Stones
Song: Baby Don't Stop, Artist: NCT U
Song: Hello, Artist: Zooey Deschanel
Song: Nerve, Artist: Katatonia
Song: I Put a Spell on You, Artist: Annie Lennox
Song: Rebellion (Lies), Artist: Arcade Fire
Song: Your Decision, Artist: Alice in Chains
Song: Techno toujours pareil, Artist: Salut C'est Cool
Song: I'm Not a Punk, Artist: Descendents
Song: A Lament For My Suzanne, Artist: 